In [2]:
# import pckages
import os
import glob
import fileinput
import shutil
from __future__ import print_function
from keras.preprocessing.image import load_img, img_to_array
from scipy.misc import imsave
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse
from keras.applications import vgg19
from keras import backend as K
import sys

Using TensorFlow backend.


In [11]:
# function definitions
#preprocess
def preprocess_image(image_path):
    #读入图像，并转化为目标尺寸。
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0) #3
    #vgg提供的预处理，主要完成（1）去均值（2）RGB转BGR（3）维度调换三个任务。
    img = vgg19.preprocess_input(img)
    return img

#post process
def deprocess_image(x):
    if K.image_dim_ordering() == 'th':
        x = x.reshape((3, img_nrows, img_ncols))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((img_nrows, img_ncols, 3))
   
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

#correlation with itself
def gram_matrix(x):
    assert K.ndim(x) == 3
    if K.image_data_format() == 'channels_first':
        features = K.batch_flatten(x)
    else:
        features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

#style gram matrix
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

#content matrix
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

# variation loss
def total_variation_loss(x):
    assert K.ndim(x) == 4
    if K.image_data_format() == 'channels_first':
        a = K.square(x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, 1:, :img_ncols - 1])
        b = K.square(x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, :img_nrows - 1, 1:])
    else:
        a = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
        b = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

#loss anf gradient
def eval_loss_and_grads(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((1, 3, img_nrows, img_ncols))
    else:
        x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

#evaluator class
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

In [ ]:
# define args
paser = parser = argparse.ArgumentParser(description='Neural style transfer with Keras.')
parser.add_argument('--base_image_path',  type=str, 
                    default='train_image/ABS03_Red_Balloon.jpg', required=False,
                    help='Path to the image to transform.')
parser.add_argument('--style_reference_image_path', default='train_image/ILU06_oil-painting.jpg', 
                    type=str, required=False,
                    help='Path to the style reference image.')
parser.add_argument('--result_prefix', default='renoir_style_transfer', type=str, required=False,
                    help='Prefix for the saved results.')
parser.add_argument('--iter', type=int, default=10, required=False,
                    help='Number of iterations to run.')
parser.add_argument('--content_weight', type=float, default=0.025, required=False,
                    help='Content weight.')
parser.add_argument('--style_weight', type=float, default=1.0, required=False,
                    help='Style weight.')
parser.add_argument('--tv_weight', type=float, default=1.0, required=False,
                    help='Total Variation weight.')

In [3]:
path_s = '.\\train_image\\'
prefixed = [filename for filename in os.listdir(path_s)]
print(len(prefixed))
path_t = '.\\test_image\\'
train = [filename for filename in os.listdir(path_t)]
print(train)

40
['alice_millar_chapel.jpg', 'arch.jpg', 'church.jpg', 'commencement.jpg', 'deering.jpg', 'Grant_DeVolson_Wood.jpg', 'robert downey.jpg']


In [ ]:
for train_base in train:
    base_folder = train_base.rstrip(".jpg") + '-all_40_styles'
    # create folder to save all images related to the training base image
    os.makedirs(base_folder, exist_ok=True)
    file_name = base_folder + '\\' + base_folder + ".txt"
    # text file to record all loss values
    fp = open(file_name, "w")
    for style_pic in prefixed:
        # new folder under the parent folder: for a specific style image
        new_folder = base_folder + '\\'+ train_base.rstrip(".jpg") + '-with_' + style_pic.split('_')[0] + '_style'
        os.makedirs(new_folder, exist_ok=True)
        
        base_dir = path_t + train_base
        args = parser.parse_args(['--base_image_path', base_dir])
        base_image_path = args.base_image_path
#         print("base path", base_image_path)
        style_reference_image_path = path_s + style_pic
        result_prefix = style_pic.split('_')[0] + '_style'
        iterations = args.iter

        # these are the weights of the different loss components
        total_variation_weight = args.tv_weight
        style_weight = args.style_weight
        content_weight = args.content_weight

        # dimensions of the generated picture.
        width, height = load_img(base_image_path).size
        img_nrows = 400
        img_ncols = int(width * img_nrows / height)

        base_image = K.variable(preprocess_image(base_image_path))
        style_reference_image = K.variable(preprocess_image(style_reference_image_path))

        if K.image_dim_ordering() == 'th':
            combination_image = K.placeholder((1, 3, img_nrows, img_ncols))
        else:
            combination_image = K.placeholder((1, img_nrows, img_ncols, 3))


        input_tensor = K.concatenate([base_image, style_reference_image, combination_image], axis=0)

        model = vgg19.VGG19(input_tensor=input_tensor,
                            weights='imagenet', include_top=False)
        print('Model loaded.')

        # get the symbolic outputs of each "key" layer (we gave them unique names).
        outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

        loss = K.variable(0.)
        layer_features = outputs_dict['block5_conv2']
        base_image_features = layer_features[0, :, :, :]
        combination_features = layer_features[2, :, :, :]
        loss += content_weight * content_loss(base_image_features,
                                              combination_features)

        feature_layers = ['block1_conv1', 'block2_conv1',
                          'block3_conv1', 'block4_conv1',
                          'block5_conv1']
        for layer_name in feature_layers:
            layer_features = outputs_dict[layer_name]
            style_reference_features = layer_features[1, :, :, :]
            combination_features = layer_features[2, :, :, :]
            sl = style_loss(style_reference_features, combination_features)
            loss += (style_weight / len(feature_layers)) * sl
        loss += total_variation_weight * total_variation_loss(combination_image)

        grads = K.gradients(loss, combination_image)

        outputs = [loss]
        if isinstance(grads, (list, tuple)):
            outputs += grads
        else:
            outputs.append(grads)
        f_outputs = K.function([combination_image], outputs)
        evaluator = Evaluator()

        x = preprocess_image(base_image_path)
        # begin to record the losses
        fp.write(train_base.rstrip(".jpg") + '-with_' + style_pic.split('_')[0] + '_style' + '\n')
        for i in range(5):
            print('Start of iteration', i, train_base)
            start_time = time.time()
            x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                             fprime=evaluator.grads, maxfun=20)
#             print('Current loss value:', min_val)
            # save current generated image to the new folder
            img = deprocess_image(x.copy())
            fname = '.\\' + new_folder + '\\' + result_prefix + '_at_iteration_%d.png' % i
            imsave(fname, img)
            end_time = time.time()
        #     print('Image saved as', fname)
        #     print('Iteration %d completed in %ds' % (i, end_time - start_time))
            # write the iteration and loss value
            fp.write('iteration' + str(i) + '\n')
            fp.write('Current loss value:' + str(min_val) + '\n')
#             fp.write('Iteration %d completed in %ds \n' % (i, end_time - start_time))
    fp.close()
    

Model loaded.
Start of iteration 0 alice_millar_chapel.jpg
Start of iteration 1 alice_millar_chapel.jpg
Start of iteration 2 alice_millar_chapel.jpg
Start of iteration 3 alice_millar_chapel.jpg
Start of iteration 4 alice_millar_chapel.jpg
Model loaded.
Start of iteration 0 alice_millar_chapel.jpg
Start of iteration 1 alice_millar_chapel.jpg
Start of iteration 2 alice_millar_chapel.jpg
Start of iteration 3 alice_millar_chapel.jpg
Start of iteration 4 alice_millar_chapel.jpg
Model loaded.
Start of iteration 0 alice_millar_chapel.jpg
Start of iteration 1 alice_millar_chapel.jpg
Start of iteration 2 alice_millar_chapel.jpg
Start of iteration 3 alice_millar_chapel.jpg
Start of iteration 4 alice_millar_chapel.jpg
Model loaded.
Start of iteration 0 alice_millar_chapel.jpg
Start of iteration 1 alice_millar_chapel.jpg
Start of iteration 2 alice_millar_chapel.jpg
Start of iteration 3 alice_millar_chapel.jpg
Start of iteration 4 alice_millar_chapel.jpg
Model loaded.
Start of iteration 0 alice_mil